## 재정정보 AI 검색 알고리즘 경진대회 코드
대회 주소 : https://dacon.io/competitions/official/236295/overview/description
### 시스템 환경
OS : Docker - pytorch/pytorch:2.3.1-cuda12.1-cudnn8-runtime <br>
CPU : Xeon® E5-2698 v4 <br> 
RAM : 97GB <br>
GPU : Q RTX 8000(VRAM 45GB) x 2 <br>
<br>
사용 모델 : rtzr/ko-gemma-2-9b-it, intfloat/multilingual-e5-large, Dongjin-kr/ko-reranker  <br>

### 제약사항 : 유료 API 전부 사용 불가능



In [22]:
!pip install transformers==4.44.0

In [2]:
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes

!pip install datasets
!pip install langchain
!pip install langchain_community
!pip install PyMuPDF
!pip install sentence-transformers
!pip install faiss-cpu
!pip install kiwipiepy rank_bm25 openai tiktoken konlpy pdfplumber
!pip install -U langchain-experimental # 최신 버전이 필요합니다 (멀티 모달을 위해)
!pip install langchain-teddynote pymupdf pymupdf4llm

In [3]:
!sudo apt update -y
!sudo apt install openjdk-11-jdk -y

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
# from dotenv import load_dotenv

# # API 키 정보 로드
# load_dotenv()

import json
import os
import unicodedata

import torch
import random
from transformers import set_seed

# 시드 설정
seed = 42
random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
set_seed(seed)

import pandas as pd
from tqdm import tqdm
import pymupdf
import pymupdf4llm
from collections import Counter

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig,
    AwqConfig
)
from accelerate import Accelerator

# Langchain 관련
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS, Chroma
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownTextSplitter, MarkdownHeaderTextSplitter
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_teddynote.retrievers import KiwiBM25Retriever,OktBM25Retriever
from langchain.retrievers import EnsembleRetriever, MultiQueryRetriever
from langchain.storage import InMemoryStore
from langchain.retrievers import ParentDocumentRetriever


from langchain.document_loaders import PDFPlumberLoader, PyMuPDFLoader, PyPDFLoader, UnstructuredPDFLoader
# from peft import PeftModel

import os
import glob
import re
from kiwipiepy import Kiwi

c:\Users\Administer\anaconda3\envs\aa\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class Opt:
    def __init__(self):
        
        # 선택된 모델
        self.base_directory = ""

        self.llm_model = os.path.join(self.base_directory, "rtzr")
        # self.llm_model_config = self.model_configs[self.llm_model]
        self.llm_peft = False
        self.llm_peft_checkpoint = "rtzr/ko-gemma-2-9b-it"
        
        self.embed_models = ["intfloat/multilingual-e5-base", "jhgan/ko-sbert-nli", "intfloat/multilingual-e5-large"]
        self.embed_model = self.embed_models[2]
        
        self.pdf_loader= "pymupdf-remaster"
        
        self.train_csv_path = os.path.join(self.base_directory, "train.csv")
        self.test_csv_path = os.path.join(self.base_directory, "test.csv")
        self.chunk_size = 512
        self.chunk_overlap = 32
        
        self.ensemble = True
        self.bm25_w = 0.3
        self.faiss_w = 0.7
        
        self.is_submit = True
        
        self.eval_sum_mode = False
        
        self.output_dir = "test_results"
        self.output_csv_file = f"{self.llm_model}_{self.embed_model.split('/')[1]}_pdf{self.pdf_loader}_chks{self.chunk_size}_chkovp{self.chunk_overlap}_bm25{self.bm25_w}_faiss{self.faiss_w}_mix_submission.csv"
        
        os.makedirs(self.output_dir, exist_ok=True)
        
    def to_json(self):
        return json.dumps(self.__dict__)
    
        
args=Opt()

In [3]:
def setup_llm_pipeline():
    tokenizer = AutoTokenizer.from_pretrained(args.llm_model)
    tokenizer.use_default_system_prompt = False
    base_directory = ""
    llm_model = os.path.join(base_directory, "rtzr")
    print(tokenizer.eos_token, tokenizer.eos_token_id)

    max_memory_mapping = {0: "14GB", 1: "45GB"}
    model = AutoModelForCausalLM.from_pretrained(
        llm_model,
        device_map="auto",
        trust_remote_code=True,
        max_memory=max_memory_mapping
        # attn_implementation="flash_attention_2",
    )
    
    text_generation_pipeline = pipeline(
        model=adapter_model if args.llm_peft else model,
        tokenizer=tokenizer,
        task="text-generation",
        return_full_text=False,
        max_new_tokens=512,
        temperature=0.7,
        top_p=0.7,
        top_k=50,
        do_sample=True
    )

    return HuggingFacePipeline(pipeline=text_generation_pipeline)

In [6]:
llm = setup_llm_pipeline()

KeyboardInterrupt: 

In [8]:
def clean_text(text):
    # 연속된 공백을 하나의 공백으로 대체
    pattern = r'\x07'
    pattern2 = r'\x03'
    # 줄 끝의 하이픈으로 끊긴 단어 연결
    text = re.sub(r'(\w+)-\s*\n\s*(\w+)', r'\1\2', text)
    # 줄바꿈 문자를 공백으로 대체 (옵션)
    text = text.replace('', '')
    text = text.replace('□', '')
    text = text.replace('ㅇ', '')
    cleaned_text = re.sub(pattern, '', text)
    cleaned_text = re.sub(pattern2, '', cleaned_text)
    text_compact = re.sub(r'\s+', ' ', cleaned_text).replace(' ', ' ')
    
    return text_compact.strip()

In [9]:
def normalize_path(path):
    return unicodedata.normalize('NFC', path)
    
def format_docs(docs):
    return "\n".join([doc.page_content for doc in docs])

def process_pdf(file_path,title):
    md_text = pymupdf4llm.to_markdown(file_path)
    
    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
        ("####", "Header 4"),
    ]

    md_header_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=False)
    md_chunks = md_header_splitter.split_text(md_text)
    dd=[]
    kiwi=Kiwi()
    for i in md_chunks:
        sp = kiwi.split_into_sents(i.page_content)
        for j in sp:
            print(clean_text(j.text))
            dd.append(Document(page_content=clean_text(j.text)))

    return dd



def create_vector_db(chunks, model_path,cnt, method='faiss'):
    model_kwargs = {'device': 'cpu'}
    encode_kwargs = {'normalize_embeddings': True}
    embeddings = HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    db = FAISS.from_documents(chunks, embedding=embeddings)
    
    return db


In [10]:
def process_pdfs_from_dataframe(df, base_directory):
    """딕셔너리에 pdf명을 키로해서 DB, retriever 저장"""
    pdf_databases = {}
    unique_paths = df['Source_path'].unique()
    cnt=0
    for path in tqdm(unique_paths, desc="Processing PDFs"):
        # 경로 정규화 및 절대 경로 생성
        normalized_path = normalize_path(path)
        full_path = os.path.normpath(os.path.join(base_directory, normalized_path.lstrip('./'))) if not os.path.isabs(normalized_path) else normalized_path
        
        pdf_title = os.path.splitext(os.path.basename(full_path))[0]
        print(f"Processing {pdf_title}...")
        
        # PDF 처리 및 벡터 DB 생성
        chunks = process_pdf(full_path,pdf_title)

        db = create_vector_db(chunks, model_path=args.embed_model,cnt=cnt)
            
        Okt_bm25_retriever = OktBM25Retriever.from_documents(chunks)
        Okt_bm25_retriever.k = 5
        faiss_retriever = db.as_retriever(search_kwargs={'k': 5},)
 
        retriever = EnsembleRetriever(
            retrievers=[Okt_bm25_retriever, faiss_retriever],
            weights=[args.bm25_w, args.faiss_w],
            search_type="mmr",
            search_kwargs={'k': 5},
        )

        # 결과 저장
        pdf_databases[pdf_title] = {
            'db': db,
            'retriever': retriever
        }
    cnt+=1
        
    return pdf_databases

In [11]:
base_directory = args.base_directory  # Your Base Directory
train_df = pd.read_csv(args.train_csv_path)
test_df = pd.read_csv(args.test_csv_path)
train_pdf_databases = None
test_pdf_databases = None
if args.is_submit:
    test_pdf_databases = process_pdfs_from_dataframe(test_df, base_directory)
else:
    train_pdf_databases = process_pdfs_from_dataframe(train_df, base_directory)

Processing PDFs:   0%|          | 0/9 [00:00<?, ?it/s]

Processing 중소벤처기업부_혁신창업사업화자금(융자)...
## 사 업 명 혁신창업사업화자금(융자) (5152-301)
## 1. 사업 코드 정보 |구분|기금|소관|실국(기관)|계정|분야|부문| |---|---|---|---|---|---|---| |코드|중소벤처기업 창업 및 진흥기금|중소벤처기업부|중소기업정책실||110|118| |명칭|||||산업·중소기업 및 에너지|창업 및 벤처| |구분|프로그램|단위사업|세부사업| |---|---|---|---| |코드|5100|5152|301| |명칭|창업환경조성|창업기업지원융자(기금)|혁신창업사업화자금(융자)|
## 구분 기금 소관 실국(기관) 계정 분야 부문 코드 110 118 중소벤처기업 중소벤처기업부 중소기업정책실 산업·중소기업
## 명칭 창업 및 진흥기금 창업 및 벤처 및 에너지
## 구분 프로그램 단위사업 세부사업 코드 5100 5152 301 명칭 창업환경조성 창업기업지원융자(기금) 혁신창업사업화자금(융자) 2. 사업 지원 형태 및 지원율 |직접|출자|출연|보조|융자|국고보조율(%)|융자율 (%)| |---|---|---|---|---|---|---| ||||○|○|||
## 직접 출자 출연 보조 융자 국고보조율(%) 융자율 (%) ○ ○ 3. 지출계획 총괄표 (단위: 백만원, %) |사업명|2022년 결산|2023년|Col4|2024년|증감 (B-A) (B-A)/A|Col7| |---|---|---|---|---|---|---| |||당초(A)|수정|확정(B)||(B-A)/A| |혁신창업사업화자 금(융자)|2,300,000|2,230,000|2,330,000|2,007,800|△222,200|△9.96| 2023년 2024년 2022년 증감 사업명 결산 당초(A) 수정 확정(B) (B-A) (B-A)/A 혁신창업사업화자 2,300,000 2,230,000 2,330,000 2,007,800 △222,200 △9.96 금(융자)
## 4. 사업목적 (창업기반지원) 기술력과 사업성이 우수하고 미래 성장가

c:\Users\Administer\anaconda3\envs\aa\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\Administer\anaconda3\envs\aa\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Administer\.cache\huggingface\hub\models--intfloat--multilingual-e5-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. F

Processing 보건복지부_부모급여(영아수당) 지원...
# 사 업 명 (67) 부모급여(영아수당) 지원 (3143-300)
# 1. 사업 코드 정보 |구분|회계|소관|실국(기관)|계정|분야|부문| |---|---|---|---|---|---|---| |코드|11 일반회계|23 보건복지부|인구정책실 보육정책관||080|08A| |명칭|||||사회복지|아동·보육| |구분 코드 명칭|회계 소관 11 일반회계 23 보건복지부|실국(기관) 계정 인구정책실 보육정책관|분야 부문 080 08A 사회복지 아동·보육| |---|---|---|---| |구분|프로그램|단위사업|세부사업| |코드|3100|3143|300| |명칭|보육지원강화|부모급여(영아수당) 지원|부모급여(영아수당) 지원| 2. 사업 지원 형태 및 지원율 |직접|출자|출연|보조|융자|국고보조율(%)|융자율 (%)| |---|---|---|---|---|---|---| ||||○||서울 25~45% 지방 55~75%|| 3. 예산 총괄표 |Col1|Col2|Col3|Col4|Col5|(단위: 백만원, %)|Col7| |---|---|---|---|---|---|---| |사업명|2022년 결산|2023년 예산 본예산(A)|2024년||증감 (B-A) (B-A)/A|| ||||정부안|확정(B)||(B-A)/A| |부모급여(영아수당) 지원|341,751|1,621,454|2,888,694|2,888,694|1,267,240|78.2| 2022년 2023년 예산 2024년 증감 사업명 결산 본예산(A) (B-A) 정부안 확정(B) (B-A)/A 부모급여(영아수당) 지원 341,751 1,621,454 2,888,694 2,888,694 1,267,240 78.2 4. 사업목적·내용 (단위: 백만원, %) (부모급여 지원) 출산 및 양육으로 손실되는 소득을 보전하고, 주 양육자의 직접돌봄이 중요한 아동발달의 특성에 따라 영아기 돌봄을 두텁게 지원하기 위해 부모급여 지급 - 영아수당을부모급여로변경‧확대 ‧ -----

Processing PDFs:  11%|█         | 1/9 [03:26<27:30, 206.28s/it]


KeyboardInterrupt: 

In [17]:
df = test_df if args.is_submit else train_df
pdf_databases = test_pdf_databases if args.is_submit else train_pdf_databases

In [18]:
def normalize_string(s):
    """유니코드 정규화"""
    return unicodedata.normalize('NFC', s)

In [19]:
from langchain_core.prompts import format_document
from langchain_community.document_transformers import LongContextReorder
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_core.runnables import RunnableMap

In [20]:
# 기본 문서 프롬프트를 생성합니다. (source, metadata 등을 추가할 수 있습니다)
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(
    template="{page_content}]"
)

def combine_documents(
    docs,  # 문서 목록
    # 문서 프롬프트 (기본값: DEFAULT_DOCUMENT_PROMPT)
    document_prompt=DEFAULT_DOCUMENT_PROMPT,
    document_separator="\n",  # 문서 구분자 (기본값: 두 개의 줄바꿈)
):
    # context 에 입력으로 넣기 위한 문서 병합
    doc_strings = [
        f"[{i}] {format_document(doc, document_prompt)}" for i, doc in enumerate(docs)
    ]  # 각 문서를 주어진 프롬프트로 포맷팅하여 문자열 목록 생성
    return document_separator.join(
        doc_strings
    )  # 포맷팅된 문서 문자열을 구분자로 연결하여 반환


def reorder_documents(docs):
    # 재정렬
    # reordering = LongContextReorder()
    # reordered_docs = reordering.transform_documents(docs)
    combined = combine_documents(docs, document_separator="\n")
    return combined

In [21]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

model = HuggingFaceCrossEncoder(model_name="Dongjin-kr/ko-reranker")

In [ ]:
# 결과를 저장할 리스트 초기화
results = []

# DataFrame의 각 행에 대해 처리
for _, row in tqdm(df.iterrows(), total=len(df), desc="Answering Questions"):
    # 소스 문자열 정규화
    source = normalize_string(row['Source'])
    question = row['Question']

    # 정규화된 키로 데이터베이스 검색
    normalized_keys = {normalize_string(k): v for k, v in pdf_databases.items()}
    retriever = normalized_keys[source]['retriever']
    compressor = CrossEncoderReranker(model=model, top_n=5)
    compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor, base_retriever=retriever)

    template = """
    {question}

    {context}

    주어진 질문에만 답변하세요.
    
    답변:
    """
    
    prompt = PromptTemplate.from_template(template)
   
    # RAG 체인 정의
    rag_chain = (
        {"context": compression_retriever | reorder_documents, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    # 답변 추론
    print(f"Question: {question}")
    full_response = rag_chain.invoke(question)
    
    print(f"Answer: {full_response}\n")

    # 결과 저장
    results.append({
        "Source": row['Source'],
        "Source_path": row['Source_path'],
        "Question": question,
        "Answer": full_response
    })

Answering Questions:   0%|          | 0/98 [00:00<?, ?it/s]

Question: 2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?


Answering Questions:   1%|          | 1/98 [00:12<20:55, 12.94s/it]

Answer: 2022년 혁신창업사업화자금(융자)의 예산은 2,300,000 백만원 입니다. 




Question: 중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?


Answering Questions:   2%|▏         | 2/98 [00:42<36:16, 22.67s/it]

...

Answer: 2023년 에너지바우처 사업 예산에서 주택관리공단 운영지원에 얼마가 할당되었는지에 대한 답변은 2023년 예산 및 2024년 예산안에서 찾아볼 수 있습니다. 

    2023년 예산안에서  '○ 주택관리공단 운영지원'은 사업운영비 1,349백만원에 포함되어 있습니다. 

    2024년 예산안에서도 '사업운영비' 항목에 주택관리공단 운영지원이 포함되어 1,872백만원으로 예산이 확정되었습니다.

    따라서 2023년에는 50백만원, 2024년에는 50백만원이 주택관리공단 운영지원에 할당되었습니다.





Question: 에너지바우처 사업의 향후 기대효과는 무엇인가요?


Answering Questions:  35%|███▍      | 34/98 [17:09<40:35, 38.05s/it]

Answer: 에너지바우처 사업의 향후 기대효과는 저소득층의 적정수준 에너지 접근성이 높아지고, 취약계층의 에너지 비용 부담 경감(약 115만 세대, 세대당 평균 36.7만원 내외)을 기대할 수 있습니다. 




Question: 에너지바우처 사업에 대한 예비타당성조사를 어떤 조사기관이 수행했나요?


Answering Questions:  36%|███▌      | 35/98 [17:36<36:28, 34.73s/it]

Answer: 에너지바우처 사업에 대한 예비타당성조사를 어떤 조사기관이 수행했나요?

    KDI(한국개발연구원 공공투자관리센터)가 2014년도 에너지바우처 지원사업(예비타당성조사 보고서)을 수행했습니다. 





Question: 21년 국정감사에서 에너지 바우처 사업에 대한 주요 지적사항은 무엇이었나요?


In [3]:
# 제출용 샘플 파일 로드
base_directory = ""
submit_df = pd.read_csv(os.path.join(base_directory, "sample_submission.csv") )

# 생성된 답변을 제출 DataFrame에 추가
submit_df['Answer'] = [item['Answer'] for item in results]
submit_df['Answer'] = submit_df['Answer'].fillna("데이콘")     # 모델에서 빈 값 (NaN) 생성 시 채점에 오류가 날 수 있음 [ 주의 ]

# 결과를 CSV 파일로 저장
submit_df.to_csv(args.output_csv_file, encoding='UTF-8-sig', index=False)

NameError: name 'results' is not defined